In [1]:
import numpy as np
import pandas as pd
import json
import datetime

import re
import nltk
from nltk.corpus import stopwords

from collections import Counter

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure


import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score

In [4]:
df = pd.read_csv("US_youtube_trending_data.csv")
df.head()

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11T19:20:14Z,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12T00:00:00Z,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,5855,35313,https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...
1,M9Pmf9AB4Mo,Apex Legends | Stories from the Outlands – “Th...,2020-08-11T17:00:10Z,UC0ZV6M2THA81QT9hrVWJG3A,Apex Legends,20,2020-08-12T00:00:00Z,Apex Legends|Apex Legends characters|new Apex ...,2381688,146739,2794,16549,https://i.ytimg.com/vi/M9Pmf9AB4Mo/default.jpg,False,False,"While running her own modding shop, Ramya Pare..."
2,J78aPJ3VyNs,I left youtube for a month and THIS is what ha...,2020-08-11T16:34:06Z,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,24,2020-08-12T00:00:00Z,jacksepticeye|funny|funny meme|memes|jacksepti...,2038853,353787,2628,40221,https://i.ytimg.com/vi/J78aPJ3VyNs/default.jpg,False,False,I left youtube for a month and this is what ha...
3,kXLn3HkpjaA,XXL 2020 Freshman Class Revealed - Official An...,2020-08-11T16:38:55Z,UCbg_UMjlHJg_19SZckaKajg,XXL,10,2020-08-12T00:00:00Z,xxl freshman|xxl freshmen|2020 xxl freshman|20...,496771,23251,1856,7647,https://i.ytimg.com/vi/kXLn3HkpjaA/default.jpg,False,False,Subscribe to XXL → http://bit.ly/subscribe-xxl...
4,VIUo6yapDbc,Ultimate DIY Home Movie Theater for The LaBran...,2020-08-11T15:10:05Z,UCDVPcEbVLQgLZX0Rt6jo34A,Mr. Kate,26,2020-08-12T00:00:00Z,The LaBrant Family|DIY|Interior Design|Makeove...,1123889,45802,964,2196,https://i.ytimg.com/vi/VIUo6yapDbc/default.jpg,False,False,Transforming The LaBrant Family's empty white ...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241387 entries, 0 to 241386
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   video_id           241387 non-null  object
 1   title              241387 non-null  object
 2   publishedAt        241387 non-null  object
 3   channelId          241387 non-null  object
 4   channelTitle       241387 non-null  object
 5   categoryId         241387 non-null  int64 
 6   trending_date      241387 non-null  object
 7   tags               241387 non-null  object
 8   view_count         241387 non-null  int64 
 9   likes              241387 non-null  int64 
 10  dislikes           241387 non-null  int64 
 11  comment_count      241387 non-null  int64 
 12  thumbnail_link     241387 non-null  object
 13  comments_disabled  241387 non-null  bool  
 14  ratings_disabled   241387 non-null  bool  
 15  description        236957 non-null  object
dtypes: bool(2), int64(5)

In [7]:
##remove the useless contribute
df.drop(columns=['video_id','thumbnail_link'],inplace=True, errors='ignore') 


In [8]:
with open("US_category_id.json") as f: categoryID = json.load(f)
categoryID, l_c = categoryID['items'], len(categoryID['items'])
ID_to_Category = dict(zip([int(categoryID[i]['id']) for i in range(l_c)],
                          [categoryID[i]['snippet']['title'] for i in range(l_c)]))

X = df[['categoryId', 'likes', 'dislikes', 'comment_count']]
y = df['view_count']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

feature_importances = model.feature_importances_
print(f"Feature importances: {feature_importances}")


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df[['likes', 'dislikes', 'comment_count']]
y = df['view_count'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


!pip install xgboost
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
xgb_model = XGBRegressor(random_state=42)
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5, 6],
    'min_child_weight': [1, 2, 3]
}
grid_search = GridSearchCV(xgb_model, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)
best_xgb_model = grid_search.best_estimator_
y_pred = best_xgb_model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print("Best parameters:", grid_search.best_params_)
print("Mean Squared Error:", mse)


In [22]:

df['engagement_rate'] = (df['likes'] + df['dislikes'] + df['comment_count']) / (df['view_count'] + 1e-6)
df['like_dislike_ratio'] = df['likes'] / (df['dislikes'] + 1e-6)
df['comment_view_ratio'] = df['comment_count'] / (df['view_count'] + 1e-6)
df['dislikes_per_comment'] = df['dislikes'] / (df['comment_count'] + 1e-6)
df['publishedAt'] = pd.to_datetime(df['publishedAt'])
df['trending_date'] = pd.to_datetime(df['trending_date'], format='%Y-%m-%d')
df['days_since_publication'] = (df['trending_date'] - df['publishedAt']).dt.days
df['likes_per_day'] = df['likes'] / (df['days_since_publication'] + 1e-6)
df['comments_per_day'] = df['comment_count'] / (df['days_since_publication'] + 1e-6)
df['view_velocity'] = np.log(df['view_count'] + 1) / (df['days_since_publication'] + 1)
print(df.head(2))


                                               title  \
0                 I ASKED HER TO BE MY GIRLFRIEND...   
1  Apex Legends | Stories from the Outlands – “Th...   

                publishedAt                 channelId  channelTitle  \
0 2020-08-11 19:20:14+00:00  UCvtRTOMP2TqYqu51xNrqAzg      Brawadis   
1 2020-08-11 17:00:10+00:00  UC0ZV6M2THA81QT9hrVWJG3A  Apex Legends   

   categoryId             trending_date  \
0          22 2020-08-12 00:00:00+00:00   
1          20 2020-08-12 00:00:00+00:00   

                                                tags  view_count   likes  \
0  brawadis|prank|basketball|skits|ghost|funny vi...     1514614  156908   
1  Apex Legends|Apex Legends characters|new Apex ...     2381688  146739   

   dislikes  ...  ratings_disabled  \
0      5855  ...             False   
1      2794  ...             False   

                                         description  engagement_rate  \
0  SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...         0.130777 

X_copy = X.copy()
X_copy.replace([np.inf, -np.inf], np.nan, inplace=True)
X_copy.fillna(X_copy.median(), inplace=True)

y = df['view_count']  
X_train, X_test, y_train, y_test = train_test_split(X_copy, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


xgb_model = XGBRegressor(random_state=42)
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5, 6],
    'min_child_weight': [1, 2, 3]
}
grid_search = GridSearchCV(xgb_model, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)
best_xgb_model = grid_search.best_estimator_
y_pred = best_xgb_model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print("Best parameters:", grid_search.best_params_)
print("Mean Squared Error:", mse)


In [36]:
# Calculate the median of the 'dislikes_per_comment' column
median_dislikes_per_comment = df['dislikes_per_comment'].median()

# Fill missing values with the median
df['dislikes_per_comment'].fillna(median_dislikes_per_comment, inplace=True)

# Calculate the median of the 'comments_per_day' column
median_comments_per_day = df['comments_per_day'].median()

# Fill missing values with the median
df['comments_per_day'].fillna(median_comments_per_day, inplace=True)

# Calculate the median of the 'view_velocity' column
median_view_velocity = df['view_velocity'].median()

# Fill missing values with the median
df['view_velocity'].fillna(median_view_velocity, inplace=True)
